In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, torch_dtype=torch.float16, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "textminr/mistral-7b-4bit-ner")

In [7]:
eval_prompt= """### Question: Der süße Brei
Ein Märchen der Gebrüder Grimm gestaltet von Heike Weinbrecht und erschienen am 27. Jänner 1830: Es war einmal ein armes, frommes Mädchen, das lebte mit seiner Mutter allein, und sie hatten nichts mehr zu essen.\n### Answer:"""

In [9]:
eval_prompt2 = """### Question: A new exhibition traces the social history of the nation through 60 pairs of shoes Richard Brooks Sun 29 Oct 2023 13.00 CET You can always judge a man by his shoes, or so the old adage goes. And the same goes for woman, as shown in an exhibition in Winchester which promises to reveal the use and role of all sorts of footwear through the ages – and what it conveyed about the wearer’s status. Co-curator Tara McKinney Marinus says: “It’s a story of our social history, rather than footwear as fashion\n### Answer:"""

In [11]:
eval_prompt3 = """### Question: Andreas Sünder 42 / 61\n### Answer:"""

In [12]:
# eval_prompt = "### Question: In their groundbreaking 2020 study on climate change impacts, Smith and Jones (2020) highlight the urgent need for sustainable practices.\n### Answer:"
model_input = tokenizer(eval_prompt3, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100, repetition_penalty=1.15)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### Question: Andreas Sünder 42 / 61
### Answer: {
  "author": "Andreas Sünder",
  "date": "42"
}
